In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz

!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.0.1-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [2]:
#import spark packages

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DateType


# Create a SparkSession
spark = SparkSession.builder.appName("SkillDrill").getOrCreate()
from pyspark import SparkFiles

In [3]:
#Reading the textfile into memory
peterpan_df = spark.read.text("/content/peterpan.txt")

In [4]:
#Showing the first 20 lines of the text file
peterpan_df.show()

+--------------------+
|               value|
+--------------------+
|All children, exc...|
|Of course they li...|
|The way Mr. Darli...|
|Mr. Darling used ...|
|Mrs. Darling was ...|
|Wendy came first,...|
|For a week or two...|
|“Now don't interr...|
|“I have one pound...|
|“Of course we can...|
|“Remember mumps,”...|
|There was the sam...|
|Mrs. Darling love...|
|No nursery could ...|
|He had his positi...|
|Nana also trouble...|
|Mrs. Darling firs...|
|I don't know whet...|
|Of course the Nev...|
|Of all delectable...|
+--------------------+
only showing top 20 rows



In [5]:
peterpan_data = spark.sparkContext.textFile("/content/peterpan.txt")
peterpan_data_split = peterpan_data.flatMap(lambda sentence: sentence.split(" "))

In [6]:
peterpan_data_count = peterpan_data_split.map(lambda word: (word, 1))

In [7]:
from operator import add
peterpan_data_count_added = peterpan_data_count.reduceByKey(add)

In [8]:
peterpan_data_count_added.saveAsTextFile("new_wordcountfile.txt")

In [9]:
spark.read.text("new_wordcountfile.txt").show()

+--------------+
|         value|
+--------------+
|  ('one,', 15)|
| ('They', 101)|
|  ('know', 64)|
|   ('way', 58)|
|  ('was', 897)|
|    ('One', 9)|
| ('when', 151)|
|   ('two', 36)|
|  ('years', 3)|
|   ('in', 623)|
|('garden,', 1)|
|('plucked', 2)|
| ('flower', 2)|
|   ('ran', 17)|
|  ('her', 361)|
|('suppose', 7)|
|  ('must', 59)|
| ('have', 243)|
|('looked', 33)|
|('rather', 40)|
+--------------+
only showing top 20 rows



In [12]:
#downloading the word file
from google.colab import files
!cp -r new_wordcountfile.txt "/content/drive/My Drive/"